In [ ]:
import os
import sys
from pathlib import Path
simFolder = str(Path(os.path.abspath('')).parent / "sim")
sys.path.append(simFolder)

c:\Users\jacob\Documents\TMS_Stimulation\sim


In [9]:
from tms_thresholds.threshold_sim import estimate_cell_threshold, cell_type_threshold_map, detect_spike
from tms_thresholds.sim_control import TMS_sim, baseline_sim
from extracellular_stim_tools import SingleExtracellular


In [ ]:
# Monophasic
tms_pulse_width_ms=0.075 # Duration of the positive phase of the pulse

pre_tms_period = 0 # For cell to reach baseline/get data on baseline activity
spike_detection_time = 1
tms_spiking_period = tms_pulse_width_ms+spike_detection_time # For spike detection

# Cell type descriptions from the Blue Brain Project glossary: https://bbp.epfl.ch/nmc-portal/glossary.html
# Layer 5 thick-tufted pyramidal cell with an early bifurcating apical tuft - continuous accommodating (adapting) pyramidal cell
cell_name = 'L5_TTPC2_cADpyr'
morphIDs = [1]
angular_resolution = 10 # Angular degrees of separation to use when when determining which field directions to simulate
                        # Applies to both the polar and azimuthal angles

duration = pre_tms_period + tms_spiking_period # ms

starting_E = 100 # Starting point of E-field amplitude in V/m to perform binary search from to estimate the cell's firing threshold at each parameter set
search_factor = 2   # Factor to scale by (up or down) when searching for the bounds to perform the binary search within
                    # Factor should be lower if guess is good to reflect confidence and minimize num of simulations
search_precision = 0.01 # Threshold estimation will be at most this amount of relative error above the true threshold (e.g. 0.01 = 1% error)

num_cores = 8 # Number of cores to allocate to the batch of parallel simulations

tms_params = dict(
    # Coupling params
    decay_rate_percent_per_mm=0,            # Rate of decay that the field diminishes by over space (uniform field at 0)
    E_field_dir={'Coord_type': 'Spherical',
                'Polar': 0,
                'Azimuthal': 0,},
    decay_dir={'Coord_type': 'Spherical',   # Direction over which the field decays (meaningless when decay is 0)
                'Polar': 180,
                'Azimuthal': 0,},
    somatodendritic_axis=[0, 1, 0],         # Defines the direction that Polar=0 points in
    # Waveform params
    stim_type='sTMS',                       # Single-pulse TMS
    stim_start_ms=pre_tms_period,           # When to apply the pulse
    num_pulses_per_burst=1,                 # Number of pulses in a burst (useful for theta burst stimulation)
    tms_pulse_width_ms=tms_pulse_width_ms,
    tms_pulse_shape='Half-Sine',
    # Simulation params
    tms_spiking_period=tms_spiking_period,  # Only pass this in for saving at the end
    simulation_duration_ms=duration,
    default_dt = 0.025,                     # Time step when outside of any pulses
    num_time_steps_in_pulse = 50,           # Number of time steps to spend on each pulse (such that pulse_dt = tms_pulse_width / num_time_steps_in_pulse)
)

# Synaptic parameters (not fully implemented)
syn_params = None

# Runs the simulations, saves the results in the data folder, and returns the threshold_map data structure
# cell_name_ID = f'{cell_name}_{morphIDs[0]}'
# estimate_cell_threshold(cell_name_ID, starting_E, search_factor, search_precision, tms_params, syn_params)
threshold_map = cell_type_threshold_map(cell_name, morphIDs, starting_E, search_factor, search_precision, angular_resolution, num_cores, tms_params, syn_params)
# print(threshold_map)

  6%|▌         | 24/422 [00:17<02:02,  3.25it/s]

: 

In [ ]:
# Monophasic
tms_pulse_width_ms=0.075 # Duration of the positive phase of the pulse
tms_pulse_width_ms *= 4 # Actual pulse width required for input of monophasic pulse (due to monophasic pulse width definition not including trailing negative phase)

pre_tms_period = 0 # For cell to reach baseline/get data on baseline activity
spike_detection_time = 1
tms_spiking_period = tms_pulse_width_ms+spike_detection_time # For spike detection

# Cell type descriptions from the Blue Brain Project glossary: https://bbp.epfl.ch/nmc-portal/glossary.html
# Layer 5 thick-tufted pyramidal cell with an early bifurcating apical tuft - continuous accommodating (adapting) pyramidal cell
cell_name_ID = 'L5_TTPC2_cADpyr_1'

duration = pre_tms_period + tms_spiking_period # ms

tms_params = dict(
    # Coupling params
    decay_rate_percent_per_mm=0,            # Rate of decay that the field diminishes by over space (uniform field at 0)
    E_field_dir={'Coord_type': 'Spherical',
                'Polar': 0,
                'Azimuthal': 0,},
    decay_dir={'Coord_type': 'Spherical',   # Direction over which the field decays (meaningless when decay is 0)
                'Polar': 180,
                'Azimuthal': 0,},
    somatodendritic_axis=[0, 1, 0],         # Defines the direction that Polar=0 points in
    # Waveform params
    stim_type='sTMS',                       # Single-pulse TMS
    efield_amplitude_V_per_m=210,
    stim_start_ms=pre_tms_period,           # When to apply the pulse
    num_pulses_per_burst=1,                 # Number of pulses in a burst (useful for theta burst stimulation)
    tms_pulse_width_ms=tms_pulse_width_ms,
    tms_pulse_shape='Monophasic',
    # Simulation params
    tms_spiking_period=tms_spiking_period,  # Only pass this in for saving at the end
    simulation_duration_ms=duration,
    default_dt = 0.025,                     # Time step when outside of any pulses
    num_time_steps_in_pulse = 50,           # Number of time steps to spend on each pulse (such that pulse_dt = tms_pulse_width / num_time_steps_in_pulse)
    # Plotting quasipotentials
    plot=False,
)

# Runs the simulations, saves the results in the data folder, and returns the threshold_map data structure
# threshold_map = cell_type_threshold_map(cell_name, morphIDs, starting_E, search_factor, search_precision, angular_resolution, num_cores, tms_params, syn_params)
# print(threshold_map)

# savestate = baseline_sim(cell_name_ID)
ecs1 = TMS_sim(cell_name_ID, tms_params)
# ecs2 = TMS_sim(cell_name_ID, tms_params, savestate=savestate)



Start time:  2025-01-22 18:45:12.324678

Creating network of 1 cell populations on 1 hosts...
  Number of cells on node 0: 1 
  Done; cell creation time = 0.09 s.
Making connections...
  Number of connections on node 0: 0 
  Done; cell connection time = 0.00 s.
  Number of stims on node 0: 0 
  Done; cell stims creation time = 0.00 s.
Recording 1 traces of 1 types on node 0
Applying extracellular stim (sTMS) to network...

Running with interval func using NEURON for 1.3 ms...
  Done; run time = 0.21 s; real-time ratio: 0.01.

Gathering data...
  Done; gather time = 0.22 s.

Analyzing...
  Cells: 1
  Connections: 0 (0.00 per cell)
  Spikes: 0 (0.00 Hz)
  Simulated time: 0.0 s; 1 workers
  Run time: 0.21 s


In [ ]:
print([a for a in ecs1.action_potentials])
print([b for b in ecs1.action_potentials_recording_ids])
print(len(ecs1.action_potentials))
print(detect_spike(ecs1))

[]
[]
0
False


In [ ]:
print([a for a in ecs2.action_potentials])
print([b for b in ecs2.action_potentials_recording_ids])
print(len(ecs2.action_potentials))
print(detect_spike(ecs2))

NameError: name 'ecs2' is not defined